# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 64 new papers today
          26 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/25 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2301.04656


extracting tarball to tmp_2301.04656...

 done.


/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:366: LatexWarning: Error parsing the document directly. Trying to recover.
  warnings.warn(LatexWarning(f"Error parsing the document directly. Trying to recover."))


...

 done.
Retrieving document from  https://arxiv.org/e-print/2301.05227


/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3442: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3442: LatexWarning: Found documentclass in tmp_2301.05118/quijotepaper_sources.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'quijote_acknow' from 'tmp_2301.05118/quijote_acknow.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'Tables/table_04' from 'tmp_2301.05118/Tables/table_04.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.16/x64/li

extracting tarball to tmp_2301.05227...

 done.


/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3442: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3442: LatexWarning: Found documentclass in tmp_2301.05227/aanda.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.04656-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.04656) | **Towards a population synthesis of discs and planets. II. Confronting  disc models and observations at the population level**  |
|| Alexandre Emsenhuber, et al. -- incl., <mark>Remo Burn</mark> |
|*Appeared on*| *2023-01-13*|
|*Comments*| *Accepted for publication in A&A*|
|**Abstract**| Aims. We want to find the distribution of initial conditions that best reproduces disc observations at the population level. Methods. We first ran a parameter study using a 1D model that includes the viscous evolution of a gas disc, dust, and pebbles, coupled with an emission model to compute the millimetre flux observable with ALMA. This was used to train a machine learning surrogate model that can compute the relevant quantity for comparison with observations in seconds. This surrogate model was used to perform parameter studies and synthetic disc populations. Results. Performing a parameter study, we find that internal photoevaporation leads to a lower dependency of disc lifetime on stellar mass than external photoevaporation. This dependence should be investigated in the future. Performing population synthesis, we find that under the combined losses of internal and external photoevaporation, discs are too short lived. Conclusions. To match observational constraints, future models of disc evolution need to include one or a combination of the following processes: infall of material to replenish the discs, shielding of the disc from internal photoevaporation due to magnetically driven disc winds, and extinction of external high-energy radiation. Nevertheless, disc properties in low-external-photoevaporation regions can be reproduced by having more massive and compact discs. Here, the optimum values of the $\alpha$ viscosity parameter lie between $3\times10^{-4}$ and $10^{-3}$ and with internal photoevaporation being the main mode of disc dispersal. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.05034-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.05034) | **How the origin of stars in the Galaxy impacts the composition of  planetary building blocks**  |
|| N. Cabral, et al. -- incl., <mark>B. Bitsch</mark> |
|*Appeared on*| *2023-01-13*|
|*Comments*| **|
|**Abstract**| Context. Our Galaxy is composed of different stellar populations with varying chemical abundances, which are thought to imprint the composition of planet building blocks (PBBs). As such, the properties of stars should affect the properties of planets and small bodies formed in their systems. In this context, high-resolution spectroscopic surveys open a window into the chemical links between and their host stars. Aims. We aim to determine the PBB composition trends for various stellar populations across the Galaxy by comparing the two large spectroscopic surveys APOGEE and GALAH. We assess the reliability of the PBB composition as determined with these surveys with a propagation error study. Methods. Stellar spectroscopic abundances from the large surveys GALAH-DR3 and APOGEE-DR17 were used as input with a stoichiometric condensation model. We classified stars into different Galactic components and we quantified the PBB composition trends as a function of [Fe/H]. We also analysed the distribution composition patterns in the [$\alpha$/Fe]-[Fe/H] diagram. Results. Our propagation error study suggests that the overall trends with [Fe/H] and [$\alpha$/Fe] are robust, which is supported by the double study of both APOGEE and GALAH. We therefore confirm the existence of a bimodal PBB composition separating the thin disc stars from the thick disc stars. Furthermore, we confirm that the stoichiometric water PBB content is anti-correlated with [Fe/H]. Conclusions. Our results imply that metal-poor stars both in the thin and thick disks are suitable hosts for water-rich PBBs and for ice-rich small bodies. However, for metal-poor stars ([Fe/H]<0), the PBBs around thick disc stars should have a higher water content than that around thin disc stars because of the $\alpha$-contentdependence of the water mass fraction. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.04659-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.04659) | **The JWST Resolved Stellar Populations Early Release Science Program II.  Survey Overview**  |
|| Daniel R. Weisz, et al. -- incl., <mark>Dustin Lang</mark>, <mark>Andrew Wetzel</mark> |
|*Appeared on*| *2023-01-13*|
|*Comments*| *23 pages, 7 Figures, 3 Tables. Submitted to AAS Journals. Comments welcome*|
|**Abstract**| We present the JWST Resolved Stellar Populations Early Release Science (ERS) science program. We obtained 27.5 hours of NIRCam and NIRISS imaging of three targets in the Local Group (Milky Way globular cluster M92, ultra-faint dwarf galaxy Draco II, star-forming dwarf galaxy WLM), which span factors of $\sim10^5$ in luminosity, $\sim10^4$ in distance, and $\sim10^5$ in surface brightness. We describe the survey strategy, scientific and technical goals, implementation details, present select NIRCam color-magnitude diagrams (CMDs), and validate the NIRCam exposure time calculator (ETC). Our CMDs are among the deepest in existence for each class of target. They touch the theoretical hydrogen burning limit in M92 ($<0.08$ $M_{\odot}$; SNR $\sim5$ at $m_{F090W}\sim28.2$; $M_{F090W}\sim+13.6$), include the lowest-mass stars observed outside the Milky Way in Draco II (0.09 $M_{\odot}$; SNR $=10$ at $m_{F090W}\sim29$; $M_{F090W}\sim+12.1$), and reach $\sim1.5$ magnitudes below the oldest main sequence turnoff in WLM (SNR $=10$ at $m_{F090W}\sim29.5$; $M_{F090W}\sim+4.6$). The PARSEC stellar models provide a good qualitative match to the NIRCam CMDs, though are $\sim0.05$ mag too blue compared to M92 F090W$-$F150W data. The NIRCam ETC (v2.0) matches the SNRs based on photon noise from DOLPHOT stellar photometry in uncrowded fields, but the ETC may not be accurate in more crowded fields, similar to what is known for HST. We release beta versions of DOLPHOT NIRCam and NIRISS modules to the community. Results from this ERS program will establish JWST as the premier instrument for resolved stellar populations studies for decades to come. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.05117-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.05117) | **QUIJOTE scientific results -- VIII. Diffuse polarized foregrounds from  component separation with QUIJOTE-MFI**  |
|| E. de la Hoz, et al. -- incl., <mark>D. Herranz</mark> |
|*Appeared on*| *2023-01-13*|
|*Comments*| *25 pages, 29 figures, Accepted for publication in MNRAS*|
|**Abstract**| We derive linearly polarized astrophysical component maps in the Northern Sky from the QUIJOTE-MFI data at 11 and 13 GHz in combination with the WMAP K and Ka bands (23 and 33 GHz) and all Planck polarized channels (30-353 GHz), using the parametric component separation method B-SeCRET. The addition of QUIJOTE-MFI data significantly improves the parameter estimation of the low-frequency foregrounds, especially the estimation of the synchrotron spectral index, $\beta_s$. We present the first detailed $\beta_s$ map of the Northern Celestial Hemisphere at a smoothing scale of $2^{\circ}$. We find statistically significant spatial variability across the sky. We obtain an average value of $-3.08$ and a dispersion of $0.13$, considering only pixels with reliable goodness-of-fit. The power law model of the synchrotron emission provides a good fit to the data outside the Galactic plane but fails to track the complexity within this region. Moreover, when we assume a synchrotron model with uniform curvature, $c_s$, we find a value of $c_s = -0.0797 \pm 0.0012$. However, there is insufficient statistical significance to determine which model is favoured, either the power law or the power law with uniform curvature. Furthermore, we estimate the thermal dust spectral parameters in polarization. Our CMB, synchrotron, and thermal dust maps are highly correlated with the corresponding products of the PR4 Planck release, although some large-scale differences are observed in the synchrotron emission. Finally, we find that the $\beta_s$ estimation in the high signal-to-noise synchrotron emission areas is prior-independent while, outside these regions, the prior governs the $\beta_s$ estimation. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.05118-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.05118) | **QUIJOTE scientific results -- IX. Radio sources in the QUIJOTE-MFI wide  survey maps**  |
|| <mark>D. Herranz</mark>, et al. |
|*Appeared on*| *2023-01-13*|
|*Comments*| *21 pages, 13 figures, 7 tables*|
|**Abstract**| We present the catalogue of Q-U-I JOint TEnerife (QUIJOTE) Wide Survey radio sources extracted from the maps of the Multi-Frequency Instrument compiled between 2012 and 2018. The catalogue contains 786 sources observed in intensity and polarization, and is divided into two separate sub-catalogues: one containing 47 bright sources previously studied by the \emph{Planck} collaboration and an extended catalogue of 739 sources either selected from the \emph{Planck} Second Catalogue of Compact Sources or found through a blind search carried out with a Mexican Hat 2 wavelet. A significant fraction of the sources in our catalogue (38.7 per cent) are within the $|b| \leq 20^\circ$ region of the Galactic plane. We determine statistical properties for those sources that are likely to be extragalactic. We find that these statistical properties are compatible with currently available models, with a $\sim$1.8 Jy completeness limit at 11 GHz. We provide the polarimetric properties of (38, 33, 31, 23) sources with P detected above the $99.99\%$ significance level at (11, 13, 17, 19) GHz, respectively. Median polarization fractions are in the $2.8$-$4.7$\% range in the 11-19 GHz frequency interval. We do not distinguish between Galactic and extragalactic sources here. The results presented here are consistent with those reported in the literature for flat- and steep-spectrum radio sources. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.05227-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.05227) | **Chrono-chemodynamical analysis of the globular cluster NGC 6355: Looking  for the fundamental bricks of the Bulge**  |
|| Stefano O. Souza, et al. -- incl., <mark>Heitor Ernandes</mark> |
|*Appeared on*| *2023-01-13*|
|*Comments*| *20 pages, 26 figures, and 9 tables. Astronomy & Astrophysics, in press*|
|**Abstract**| The information on Galactic assembly time is imprinted on the chemodynamics of globular clusters. This makes them important probes that help us to understand the formation and evolution of the Milky Way. Discerning between in-situ and ex-situ origin of these objects is difficult when we study the Galactic bulge, which is the most complex and mixed component of the Milky Way. To investigate the early evolution of the Galactic bulge, we analysed the globular cluster NGC 6355. We derived chemical abundances and kinematic and dynamic properties by gathering information from high-resolution spectroscopy with FLAMES-UVES, photometry with the Hubble Space Telescope, and Galactic dynamic calculations applied to the globular cluster NGC 6355. We derive an age of $13.2\pm1.1$ Gyr and a metallicity of [Fe/H]$=-1.39\pm0.08$ for NGC 6355, with $\alpha$-enhancement of [$\alpha$/Fe]$=+0.37\pm0.11$. The abundance pattern of the globular cluster is compatible with bulge field RR Lyrae stars and in-situ well-studied globular clusters. The orbital parameters suggest that the cluster is currently confined within the bulge volume when we consider a heliocentric distance of $8.54\pm0.19$ kpc and an extinction coefficient of $R_V = 2.84\pm0.02$. NGC 6355 is highly likely to come from the main bulge progenitor. {Nevertheless, it still} has a low probability of being formed from an accreted event because its age is uncertain and because of the combined [Mg/Mn] [Al/Fe] abundance. Its relatively low metallicity with respect to old and moderately metal-poor inner Galaxy clusters may suggest a low-metallicity floor for globular clusters that formed in-situ in the early Galactic bulge. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.04699-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.04699) | **CMZoom III: Spectral Line Data Release**  |
|| Daniel Callanan, et al. -- incl., <mark>Jonathan Henshaw</mark>, <mark>Qizhou Zhang</mark> |
|*Appeared on*| *2023-01-13*|
|*Comments*| *44 pages, 41 figures*|
|**Abstract**| We present an overview and data release of the spectral line component of the SMA Large Program, \textit{CMZoom}. \textit{CMZoom} observed $^{12}$CO(2-1), $^{13}$CO(2-1) and C$^{18}$O(2-1), three transitions of H$_{2}$CO, several transitions of CH$_{3}$OH, two transitions of OCS and single transitions of SiO and SO, within gas above a column density of N(H$_2$)$\ge 10^{23}$\,cm$^{-2}$ in the Central Molecular Zone (CMZ; inner few hundred pc of the Galaxy). We extract spectra from all compact 1.3\,mm \emph{CMZoom} continuum sources and fit line profiles to the spectra. We use the fit results from the H$_{2}$CO 3(0,3)-2(0,2) transition to determine the source kinematic properties. We find $\sim 90$\% of the total mass of \emph{CMZoom} sources have reliable kinematics. Only four compact continuum sources are formally self-gravitating. The remainder are consistent with being in hydrostatic equilibrium assuming that they are confined by the high external pressure in the CMZ. Based on the mass and density of virially bound sources, and assuming star formation occurs within one free-fall time with a star formation efficiency of $10\% - 75\%$, we place a lower limit on the future embedded star-formation rate of $0.008 - 0.06$\,M$_{\odot}$\,yr$^{-1}$. We find only two convincing proto-stellar outflows, ruling out a previously undetected population of very massive, actively accreting YSOs with strong outflows. Finally, despite having sufficient sensitivity and resolution to detect high-velocity compact clouds (HVCCs), which have been claimed as evidence for intermediate mass black holes interacting with molecular gas clouds, we find no such objects across the large survey area. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error Could not find figure .pdf</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.04694-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.04694) | **Science Priorities for the Extraction of the Solid MSR Samples from  their Sample Tubes**  |
|| N. Dauphas, et al. -- incl., <mark>T. Kleine</mark> |
|*Appeared on*| *2023-01-13*|
|*Comments*| *8 pages, 3 figures, 1 table, report NASA-ESA Mars Rock Team Report*|
|**Abstract**| Preservation of the chemical and structural integrity of samples that will be brought back from Mars is paramount to achieving the scientific objectives of MSR. Given our knowledge of the nature of the samples retrieved at Jezero by Perseverance, at least two options need to be tested for opening the sample tubes: (1) One or two radial cuts at the end of the tube to slide the sample out. (2) Two radial cuts at the ends of the tube and two longitudinal cuts to lift the upper half of the tube and access the sample. Strategy 1 will likely minimize contamination but incurs the risk of affecting the physical integrity of weakly consolidated samples. Strategy 2 will be optimal for preserving the physical integrity of the samples but increases the risk of contamination and mishandling of the sample as more manipulations and additional equipment will be needed. A flexible approach to opening the sample tubes is therefore required, and several options need to be available, depending on the nature of the rock samples returned. Both opening strategies 1 and 2 may need to be available when the samples are returned to handle different sample types (e.g., loosely bound sediments vs. indurated magmatic rocks). This question should be revisited after engineering tests are performed on analogue samples. The MSR sample tubes will have to be opened under stringent BSL4 conditions and this aspect needs to be integrated into the planning. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2301.04656.md
    + _build/html/tmp_2301.04656/./manara_canon_2e6.png
    + _build/html/tmp_2301.04656/./disklifetimes_canon.png
    + _build/html/tmp_2301.04656/./typical_gas_sinks.png
exported in  _build/html/2301.05034.md
    + _build/html/tmp_2301.05034/./Figures/17_Mfrac_ThinThick_Int_New.png
    + _build/html/tmp_2301.05034/./Figures/17_Mfrac_ThinThick_Ext_New.png
    + _build/html/tmp_2301.05034/./Figures/10_Mfrac_H2O_AlphaFe_FeH_External_2panels_lim.png
    + _build/html/tmp_2301.05034/./Figures/5_nMgnH_AlphaFe_FeH_Internal_2panels_lim.png
    + _build/html/tmp_2301.05034/./Figures/6_nSinH_AlphaFe_FeH_Internal_2panels_lim.png
    + _build/html/tmp_2301.05034/./Figures/7_nMgHminnSiH_AlphaFe_FeH_Internal_2panels_lim.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\paperone}{\citetalias{2022AABurnA}}$
$\newcommand{\paperthree}{\citetalias{Disk3}}$
$\newcommand{\tabletv}{Table~3}$
$\newcommand{\tablets}{Table~4}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\paperone$}{\citetalias{2022AABurnA}}$
$\newcommand{$\paperthree$}{\citetalias{Disk3}}$
$\newcommand{$\tabletv$}{Table~3}$
$\newcommand{$\tablets$}{Table~4}$</div>



<div id="title">

# Towards a population synthesis of discs and planets: Tables 3 and 4 are only available in electronic form at the CDS via anonymous ftp to cdsarc.cds.unistra.fr (130.79.128.5) or via https://cdsarc.cds.unistra.fr/cgi-bin/qcat?J/A+A/

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2301.04656-b31b1b.svg)](https://arxiv.org/abs/2301.04656)<mark>Appeared on: 2023-01-13</mark> - _Accepted for publication in A&A_

</div>
<div id="authors">

Alexandre Emsenhuber, et al. -- incl., <mark>Remo Burn</mark>

</div>
<div id="abstract">

**Abstract:** Aims. We want to find the distribution of initial conditions that best reproduces disc observations at the population level. Methods. We first ran a parameter study using a 1D model that includes the viscous evolution of a gas disc, dust, and pebbles, coupled with an emission model to compute the millimetre flux observable with ALMA. This was used to train a machine learning surrogate model that can compute the relevant quantity for comparison with observations in seconds. This surrogate model was used to perform parameter studies and synthetic disc populations. Results. Performing a parameter study, we find that internal photoevaporation leads to a lower dependency of disc lifetime on stellar mass than external photoevaporation. This dependence should be investigated in the future. Performing population synthesis, we find that under the combined losses of internal and external photoevaporation, discs are too short lived. Conclusions. To match observational constraints, future models of disc evolution need to include one or a combination of the following processes: infall of material to replenish the discs, shielding of the disc from internal photoevaporation due to magnetically driven disc winds, and extinction of external high-energy radiation. Nevertheless, disc properties in low-external-photoevaporation regions can be reproduced by having more massive and compact discs. Here, the optimum values of the $\alpha$ viscosity parameter lie between $3\times10^{-4}$ and $10^{-3}$ and with internal photoevaporation being the main mode of disc dispersal. 

</div>

<div id="div_fig1">

<img src="tmp_2301.04656/./manara_canon_2e6.png" alt="Fig4" width="100%"/>

**Figure 4. -** Histogram for stellar accretion rate vs. disc mass at \SI{2}{\mega\year} and the same synthetic disc populations shown in Fig. \ref{fig:dlt-pop-canon}. The observed data from the Lupus (in red) and Chamaeleon I (orange) star forming regions from \citet{2019AAManara} are shown for comparison. Two disc dispersal timescales $\tau=\dot{M}_\mathrm{G}/M_\mathrm{G}$ of \SI{1}{\mega\year} and \SI{10}{\mega\year}(assuming that the gas disc mass $M_\mathrm{G}$ is \num{100} times the observed dust mass) and the best fit to the data from \citet{2016AAManaraB} are shown as well. (*fig:mar-pop-canon*)

</div>
<div id="div_fig2">

<img src="tmp_2301.04656/./disklifetimes_canon.png" alt="Fig1" width="100%"/>

**Figure 1. -** Cumulative distribution of disc lifetimes for a population with canonical parameter distribution (see text). Two exponential decays following \citet{2010AAFedele} with a characteristic time of \SI{2.3}{\mega\year}(accretion) and \SI{3}{\mega\year}(infrared excess) and the results of \citet{2014AARibas} are shown as well. (*fig:dlt-pop-canon*)

</div>
<div id="div_fig3">

<img src="tmp_2301.04656/./typical_gas_sinks.png" alt="Fig3" width="100%"/>

**Figure 3. -** Evolution of the relative gas mass: in the disc (blue), accreted onto the star (green), and lost by internal (orange) or external (red) photoevaporation until disc dispersal at \SI{1.38}{\mega\year}. This represents a typical disc, with $M_\star=\SI{0.5}{\msol}$, $M_\mathrm{G}/M_\star=\num{0.1}$, $\beta=0.9$, $R_\mathrm{in}=\SI{0.1}{\au}$, $r_1=\SI{87.8}{\au}$, $\alpha=\num{1e-3}$, $L_\mathrm{X}=\SI{7.02e29}{\erg\per\second}$, and $F=\SI{10}{G_0}$. The parameters for the solid disc are irrelevant, as this shows only the gas component, except that we used $f_\mathrm{D/G}=\num{0.0149}$ to compute the solid mass needed to set the characteristic radius $r_1$. (*fig:typical*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$</div>



<div id="title">

# How the origin of stars in the Galaxy impacts the composition of planetary building blocks

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2301.05034-b31b1b.svg)](https://arxiv.org/abs/2301.05034)<mark>Appeared on: 2023-01-13</mark> - __

</div>
<div id="authors">

N. Cabral, et al. -- incl., <mark>B. Bitsch</mark>

</div>
<div id="abstract">

**Abstract:** Our Galaxy is composed of different stellar populations with varying chemical abundances, which are thought to imprint the composition of planet building blocks (PBBs). As such, the properties of stars should affect the properties of planets and small bodies formed in their systems. In this context, high-resolution spectroscopic surveys open a window into the chemical links between and their host stars.We aim to determine the PBB composition trends for various stellar populations across the Galaxy by comparing the two large spectroscopic surveys APOGEE and GALAH. We assess the reliability of the PBB composition as determined with these surveys with a propagation error study.Stellar spectroscopic abundances from the large surveys GALAH-DR3 and APOGEE-DR17 were used as input with a stoichiometric condensation model. We classified stars into different Galactic components and we quantified the PBB composition trends as a function of[Fe/H]. We also analysed the distribution composition patterns in the[$\alpha$/Fe]-[Fe/H]diagram.Our propagation error study suggests that the overall trends with[Fe/H]and[$\alpha$/Fe]are robust, which is supported by the double study of both APOGEE and GALAH. We therefore confirm the existence of a bimodal PBB composition separating the thin disc stars from the thick disc stars. Furthermore, we confirm that the stoichiometric water PBB content is anti-correlated with[Fe/H].Our results imply that metal-poor stars both in the thin and thick disks are suitable hosts for water-rich PBBs and for ice-rich small bodies. However, for metal-poor stars ([Fe/H]<0), the PBBs around thick disc stars should have a higher water content than that around thin disc stars because of the$\alpha$-content dependence of the water mass fraction. Given the importance of the initial water abundance of the PBBs in recent planet formation simulations, we expect that the star origin influences the exoplanet population properties across the Galaxy.

</div>

<div id="div_fig1">

<img src="tmp_2301.05034/./Figures/17_Mfrac_ThinThick_Int_New.png" alt="Fig11.1" width="50%"/><img src="tmp_2301.05034/./Figures/17_Mfrac_ThinThick_Ext_New.png" alt="Fig11.2" width="50%"/>

**Figure 11. -** Mean molecular mass fractions per bin of metallicity for thin disc (solid line) and thick disc (dashed line). Top panels correspond to APOGEE-DR17; bottom panels correspond to GALAH-DR3. Left panels correspond to the inner proto-planetary disc (T>150K); right panels correspond to the outer proto-planetary disc (T<150K, which includes the H2O molecule). (*Fig_Mfrac_ThinThickperFeH*)

</div>
<div id="div_fig2">

<img src="tmp_2301.05034/./Figures/10_Mfrac_H2O_AlphaFe_FeH_External_2panels_lim.png" alt="Fig3" width="100%"/>

**Figure 3. -** PBB mass fraction distribution of H$_2$O for T<150 K (outer proto-planetary disc) in the [$\alpha$/Fe]-[Fe/H] diagram. We display the surveys APOGEE-DR17 (top panels) and GALAH-DR3 (bottom panels). (*Fig7_Mfrac_AlphaFe_Outer*)

</div>
<div id="div_fig3">

<img src="tmp_2301.05034/./Figures/5_nMgnH_AlphaFe_FeH_Internal_2panels_lim.png" alt="Fig8.1" width="33%"/><img src="tmp_2301.05034/./Figures/6_nSinH_AlphaFe_FeH_Internal_2panels_lim.png" alt="Fig8.2" width="33%"/><img src="tmp_2301.05034/./Figures/7_nMgHminnSiH_AlphaFe_FeH_Internal_2panels_lim.png" alt="Fig8.3" width="33%"/>

**Figure 8. -** Stellar abundance distribution in the [$\alpha$/Fe]-[Fe/H] diagram for  Mg/H (left panel), Si/H (middle panel), and Mg/H-Si/H (right panel). We display the surveys APOGEE-DR17 (top panels) and GALAH-DR3 (bottom panels). (*Fig6_MgH_AlphaFe_FeH*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

167  publications in the last 7 days.
	 _build/html/2301.05034.md
	 _build/html/2301.04656.md
	 _build/html/2301.04533.md
	 _build/html/2301.04442.md
	 _build/html/2301.03811.md
	 _build/html/2301.03689.md
	 _build/html/2301.03387.md
	 _build/html/2301.03209.md
	 _build/html/2301.02671.md
	 _build/html/2301.02477.md
	 _build/html/2301.02313.md
	 _build/html/2301.02265.md
	 _build/html/2301.01937.md
	 _build/html/2301.01895.md
	 _build/html/2301.01684.md
	 _build/html/2301.01306.md
	 _build/html/2301.00881.md
	 _build/html/2301.00854.md
	 _build/html/2301.00578.md
	 _build/html/2212.12039.md
	 _build/html/2212.11991.md
	 _build/html/2212.11425.md
	 _build/html/2212.11177.md
	 _build/html/2212.10804.md
	 _build/html/2212.10512.md
	 _build/html/2212.09766.md
	 _build/html/2212.09675.md
	 _build/html/2212.09661.md
	 _build/html/2212.09652.md
	 _build/html/2212.09443.md
	 _build/html/2212.09307.md
	 _build/html/2212.09168.md
	 _build/html/2212.09159.md
	 _build/html/2212.08702.md
	 _build/h

In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

14  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers